In [11]:
# Import
from dotenv import load_dotenv
import os
import sys
from requests import post
import json
import time
import pandas as pd

In [7]:
load_dotenv()

client_id_igdb = os.getenv("IGDB_ID_CLIENT")
client_secret_igdb = os.getenv("IGDB_SECRET_CLIENT")

In [8]:
print(client_id_igdb)
print(client_secret_igdb)

4pmce94am1hizog6ff9feucpmtki1q
7lz6ivr2elpwktm6x3bsbs0lefp2lo


In [9]:
# Récupération du token d'accès pour IGDB

url = f"https://id.twitch.tv/oauth2/token?client_id={client_id_igdb}&client_secret={client_secret_igdb}&grant_type=client_credentials"

print(url)
response = requests.post(url)
if response.status_code == 200:
    response_json = response.json()
    if 'expires_in' in response_json == 0:
        print(f"Le token a expiré, il faut le renouveler")
    access_token = response_json['access_token']
    print(f"Access Token: {access_token}")
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)

https://id.twitch.tv/oauth2/token?client_id=4pmce94am1hizog6ff9feucpmtki1q&client_secret=7lz6ivr2elpwktm6x3bsbs0lefp2lo&grant_type=client_credentials
Access Token: 6dekox56hmsrswfyazjxp7y2cap11g


In [15]:
# Récuperer le nombres de jeux
url_igdb_engines_count = "https://api.igdb.com/v4/games/count"

headers = {
    'Client-ID': client_id_igdb,   
    'Authorization': f'Bearer {access_token}',
    }

response = post(url_igdb_engines_count, headers=headers, data='fields *;')
print(response.json())

{'count': 320427}


In [17]:
# Récupération des games


headers_game = {
    'Client-ID': client_id_igdb,
    'Authorization': f'Bearer {access_token}',
}

response = post('https://api.igdb.com/v4/games', headers=headers_game , data='fields name,age_ratings,category,dlcs,first_release_date,game_engines,game_status,genres,language_supports,multiplayer_modes,videos,websites,total_rating,total_rating,summary  ; limit 500;')
# Vérification de la réponse
if response.status_code == 200:
    # On transforme la réponse en JSON
    response_json = response.json()
    # On transforme le JSON en DataFrame
    df_games = pd.DataFrame(response_json)
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)
    
df_games

,id,category,game_engines,genres,name,summary,videos,first_release_date,websites,age_ratings,language_supports,total_rating,multiplayer_modes,game_status,dlcs
0,330684,14,[439],"[10, 33]",Nightmare Kart: The Old Karts,An upcoming free expansion to Nightmare Kart w...,[139717],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,177310,0,[750],"[13, 32]",The Undying Beast,"There was a flash of light, a choir of glass, ...",NaN,1.586736e+09,"[237634, 473902]",NaN,NaN,NaN,NaN,NaN,NaN
2,43460,3,NaN,[25],Sengoku Anthology,Sengoku Anthology is a collection of the three...,NaN,1.251763e+09,NaN,[15301],NaN,NaN,NaN,NaN,NaN
3,282079,5,NaN,"[5, 8]",Rockman X: New Year 2023,A small hack to celebrate the Chinese New Year...,NaN,1.672531e+09,[606539],NaN,[876934],NaN,NaN,NaN,NaN
4,63844,0,NaN,[14],Ace wo Nerae!,A tennis game for the Super Famicom based on t...,[43074],7.565184e+08,"[161465, 373934]",NaN,[685891],52.904629,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,285652,0,[64],"[5, 32, 33]",Notebook Wars,Get ready for the ultimate notebook shooter!\n...,[106690],1.293754e+09,[612953],NaN,"[903519, 903520, 903521]",NaN,NaN,NaN,NaN
496,17437,0,[3],"[12, 31, 32]",Divinia Chronicles: Relics of Gan-Ti,Divinia Chronicles: Relics of Gan-Ti is a sing...,NaN,1.441325e+09,"[24983, 24982, 432359]",NaN,"[398323, 398327, 398333, 398336, 398319, 39832...",70.000000,NaN,4.0,NaN
497,221619,0,[64],[34],Meet'N'Fuck Subway Story,"Subway story is an erotic, romance game.",NaN,1.214093e+09,[317807],NaN,NaN,NaN,NaN,NaN,NaN
498,200472,0,NaN,"[31, 32]",Candlelight,Ed broke the curfew to visit his friends in or...,NaN,1.652400e+09,[281956],NaN,"[553078, 553079, 553080, 553081]",NaN,NaN,NaN,NaN


In [20]:
def get_all_games():
    all_games = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/games', headers=headers_game, data=f'fields name,age_ratings,category,dlcs,first_release_date,game_engines,game_status,genres,language_supports,multiplayer_modes,videos,websites,total_rating,total_rating,summary; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            all_games.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break

    return pd.DataFrame(all_games)

In [22]:
df_games = get_all_games()

In [23]:
df_games.to_csv('../csv/games.csv', index=False)

OSError: Cannot save file into a non-existent directory: '..\csv'